In [21]:
import json

# Input and output file paths
input_file_path = "data.jsonl"
output_file_path = "transformed_data.jsonl"

# System message to be added
system_message = {"role": "system", "content": "You are a helpful assistant."}

# Read and transform the data
with open(input_file_path, 'r') as infile, open(output_file_path, 'w') as outfile:
    for line in infile:
        # Skip empty lines
        if not line.strip():
            continue
        try:
            # Load each JSON object
            data = json.loads(line.strip())

            # Create the new format
            transformed_data = {
                "messages": [
                    system_message,
                    {"role": "user", "content": data['prompt']},
                    {"role": "assistant", "content": data['completion']}
                ]
            }

            # Write the transformed data to the output file
            json.dump(transformed_data, outfile)
            outfile.write('\n')
        except json.JSONDecodeError as e:
            print(f"Skipping invalid JSON line: {line.strip()} (Error: {e})")

print("Transformation complete. Output saved to", output_file_path)


Transformation complete. Output saved to transformed_data.jsonl


In [22]:
from openai import OpenAI

In [23]:
client = OpenAI()

In [24]:
client.files.create(file=open("transformed_data.jsonl","rb"),purpose='fine-tune')

FileObject(id='file-MczA3dMybRO0jZjhcsqzo9zD', bytes=4519, created_at=1732156160, filename='transformed_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [25]:
files = client.files.list()

In [26]:
for file in files.data:
  print(file)

FileObject(id='file-MczA3dMybRO0jZjhcsqzo9zD', bytes=4519, created_at=1732156160, filename='transformed_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
FileObject(id='file-JdVhcoArRfFkjWUPmCSCCb8x', bytes=1682, created_at=1732155872, filename='data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
FileObject(id='file-hW8wtyH5r4sb5xPt7odlzeJ1', bytes=4519, created_at=1732153551, filename='transformed_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
FileObject(id='file-vzee8uhIOM68WHj0fQ1VX306', bytes=34512, created_at=1732151233, filename='step_metrics.csv', object='file', purpose='fine-tune-results', status='processed', status_details=None)
FileObject(id='file-RAG6vasOGHGFZdiIRQJVNaqO', bytes=34536, created_at=1732150929, filename='step_metrics.csv', object='file', purpose='fine-tune-results', status='processed', status_details=None)
FileObject(id='file-eF6VBNUzgTxryC1T

In [27]:
for file in files.data:
  if file.filename == 'transformed_data.jsonl':
    training_file_id = file.id

In [28]:
client.fine_tuning.jobs.create(
    training_file=training_file_id,
    model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-nz3nDhn6O0XafJTAAISRQylq', created_at=1732156185, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-KxV99padaMMP1o5sAbJTr0mG', result_files=[], seed=1941959223, status='validating_files', trained_tokens=None, training_file='file-hW8wtyH5r4sb5xPt7odlzeJ1', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [29]:
jobs=client.fine_tuning.jobs.list()

In [30]:
client.fine_tuning.jobs.retrieve('ftjob-nz3nDhn6O0XafJTAAISRQylq')

FineTuningJob(id='ftjob-nz3nDhn6O0XafJTAAISRQylq', created_at=1732156185, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=6, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-KxV99padaMMP1o5sAbJTr0mG', result_files=[], seed=1941959223, status='running', trained_tokens=None, training_file='file-hW8wtyH5r4sb5xPt7odlzeJ1', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [31]:
import openai

# Replace with your fine-tuned model ID
fine_tuned_model = 'ft:gpt-3.5-turbo-0125:personal::AVrKz1JZ'

# Questions from the dataset
questions = [
    "How do I change the air filter in the HVAC system?",
    "What is the password for the guest Wi-Fi?",
    "How can I remove pet hair from the couch?",
    "Where are the spare lightbulbs?",
    "How do I change the batteries in the remote?",
    "Where can I find the first aid kit?",
    "What time does the local grocery store close?",
    "How do I clean the oven?",
    "How do I adjust the height of the office chair?",
    "Where do I find the user manual for the washing machine?",
    "What should I do if the dishwasher is leaking?",
    "How do I turn on the outdoor sprinkler system?",
    "What should I do if the water heater is not heating?"
]

# Loop through each question to get responses from the fine-tuned model
for question in questions:
    prompt = f"Q: {question}\nA:"
    print("Prompt:", prompt)

    # Send the request to the fine-tuned model
    response = client.chat.completions.create(
        model=fine_tuned_model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant providing accurate answers to household-related queries."},
            {"role": "user", "content": prompt},
        ],
    )

    # Extract and print the completion (response structure update)
    answer = response.choices[0].message.content.strip() # This may need to be updated depending on the response format
    print(f"Q: {question}")
    print(f"A: {answer}")
    print("-" * 50)


Prompt: Q: How do I change the air filter in the HVAC system?
A:
Q: How do I change the air filter in the HVAC system?
A: Turn off the HVAC system, remove the old filter from the filter slot, and replace it with a new one. Make sure it fits properly.
--------------------------------------------------
Prompt: Q: What is the password for the guest Wi-Fi?
A:
Q: What is the password for the guest Wi-Fi?
A: Check the card next to the router for the Wi-Fi password.
--------------------------------------------------
Prompt: Q: How can I remove pet hair from the couch?
A:
Q: How can I remove pet hair from the couch?
A: Use a lint roller or a damp rubber glove to gently wipe off the pet hair from the fabric.
--------------------------------------------------
Prompt: Q: Where are the spare lightbulbs?
A:
Q: Where are the spare lightbulbs?
A: In the drawer in the hallway closet, on the left side.
--------------------------------------------------
Prompt: Q: How do I change the batteries in the re